# data_prep.py

In [34]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)


# my solution: add bias
# data['bias'] = 1


# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

/Users/rui/anaconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [35]:
type(features)

pandas.core.frame.DataFrame

In [36]:
features.head()

,gre,gpa,rank_1,rank_2,rank_3,rank_4,bias
209,-0.066657,0.289305,0,1,0,0,1
280,0.625884,1.445476,0,1,0,0,1
33,1.837832,1.603135,0,0,1,0,1
210,1.318426,-0.131120,0,0,0,1,1
93,-0.066657,-1.208461,0,1,0,0,1


In [37]:
features.values

array([[-0.06665712,  0.28930534,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.62588442,  1.44547565,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.83783211,  1.60313523,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       ..., 
       [-0.41292789, -0.57782239,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.62588442,  1.60313523,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.41292789, -0.28877981,  1.        , ...,  0.        ,
         0.        ,  1.        ]])

In [38]:
type(targets)

pandas.core.series.Series

In [39]:
targets.head()

209    0
280    0
33     1
210    0
93     0
Name: admit, dtype: int64

# gradient.py

In [40]:
# import numpy as np
# from data_prep import features, targets, features_test, targets_test

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / (n_features**0.5), size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)

    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output
        h = np.dot(weights, x)
                
        # TODO: Calculate the output
        output = sigmoid(h)

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        output_gradient = output * (1 - output)
        
        # NOTE: Error is MSE here
        error_term = error * output_gradient

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
            
        print("weights: ", weights)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.334010227431
weights:  [ 0.19406969 -0.04353004  0.24214081  0.56250567 -0.10166863 -0.09744617
  0.55896069]
Train loss:  0.199833762587
weights:  [ 0.24537091  0.25384684  0.3170125   0.04953482 -0.44801543 -0.43800167
 -0.56604076]
Train loss:  0.198159404854
weights:  [ 0.24932726  0.29419959  0.46101426  0.01028642 -0.48661856 -0.54875134
 -0.61064022]
Train loss:  0.197541235358
weights:  [ 0.24937856  0.30107568  0.5558073   0.00287167 -0.50512913 -0.6247033
 -0.61772445]
Train loss:  0.197260959295
weights:  [ 0.25035023  0.30244559  0.61660267  0.00264446 -0.51448816 -0.68059864
 -0.62241066]
Train loss:  0.197124973608
weights:  [ 0.25164401  0.3028249   0.65607704  0.00491195 -0.51847653 -0.72266872
 -0.62672725]
Train loss:  0.197054845193
weights:  [ 0.25279131  0.30296231  0.68218952  0.00798431 -0.51937551 -0.75479579
 -0.63056847]
Train loss:  0.197016698613
weights:  [ 0.25368854  0.30300473  0.69980711  0.01112283 -0.51860358 -0.77960372
 -0.63384834]
T